In [1]:
''' 임시로 엑셀 변환하는 코드 '''
# import os
# import pandas as pd
# # 변환할 CSV 파일의 경로와 이름
# csv_file = "../dataset/본문_형태소분석/2018산림복지_본문_형태소분석(형용사).csv"

# # CSV 파일을 데이터프레임으로 읽기
# dataframe = pd.read_csv(csv_file, encoding='UTF-8')

# # XLSX 파일로 변환
# xlsx_file = os.path.splitext(csv_file)[0] + '.xlsx'
# dataframe.to_excel(xlsx_file, index=False)

' 임시로 엑셀 변환하는 코드 '

In [1]:
import pandas as pd              # pandas 패키지 임포트
import numpy as np               # numpy 패키지 임포트
from konlpy.tag import Okt       # 한글 분석 패키지 konlpy 에서 Okt 형태소 분석기 임포트
from tqdm import tqdm     # for 반복문 진행상황 표시
from collections import Counter
# scikit-learn에서 제공하는 TF-IDF 생성 패키지 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()   # 코드 단순화를 위해 Okt()를 변수 okt에 할당

In [2]:
# 불용어 목록 만들기
# ※ 분석할 문장에서 조사를 뽑아 유일값만 남기면 됨.

df_josa = pd.read_csv("../dataset/2022/본문_형태소분석/7월/나무문화_7월_형태소분석(조사).csv", encoding='UTF-8')
josa_list = df_josa["조사"].tolist() # 이다, 보다는 지워야 함.
josa_list.remove("이다")
josa_list.remove("보다")
print(len(josa_list))

394


In [3]:
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_7월_형태소분석.txt", encoding='UTF-8')
df

,split_str,org_idx,morphs,pos_tag
0,25일급여 삼성카드 특판적금 서울행복신협 구즉신협 카카오뱅크8 적금 새빗켐 지대로사...,0,"['25일', '급여', '삼성', '카드', '특판', '적금', '서울', '행...","[('25일', 'Number'), ('급여', 'Foreign'), ('삼성', ..."
1,근데 급 다음주부터 발령나서 다른 곳으로 출근한다,0,"['근데', '급', '다음주', '부터', '발령', '나서', '다른', '곳'...","[('근데', 'Adverb'), ('급', 'Noun'), ('다음주', 'Nou..."
2,2 3일 전에 알려주고 갑자기 거기로 출근이야 좌천인가 걱정됨,0,"['2', '3일', '전', '에', '알려주고', '갑자기', '거기', '로'...","[('2', 'Number'), ('3일', 'Number'), ('전', 'Nou..."
3,잘 적응하라고 뇌물을 준비해야 하나 ㅋㅋ 갑자기 문자메세지로 삼성카드 5월 링크 이...,0,"['잘', '적응하라고', '뇌물', '을', '준비', '해야', '하나', 'ㅋ...","[('자다', 'Verb'), ('적응하다', 'Adjective'), ('뇌물',..."
4,소소하게 가계부 적립할게 생겨서 기분 좋은 날 2 화요일 오래전 엄마 집 옆에 있는...,0,"['소소하게', '가계부', '적립', '할게', '생겨서', '기분', '좋은',...","[('소소하다', 'Adjective'), ('가계부', 'Noun'), ('적립'..."
...,...,...,...,...
901121,현재 지하 1층에서 흙과몸 성긴 연결 촘촘한 관계 6 9 전시실 를 테마로 강요배 ...,18988,"['현재', '지하', '1', '층', '에서', '흙', '과', '몸', '성...","[('현재', 'Noun'), ('지하', 'Noun'), ('1', 'Number..."
901122,그 외에 Big Man 다시 일어서는 몸 박치호 애도 상실의 끝에서 김수자 게르하르...,18988,"['그', '외', '에', 'Big', 'Man', '다시', '일어서는', '몸...","[('그', 'Noun'), ('외', 'Noun'), ('에', 'Josa'), ..."
901123,광양시 문화관광 061 797 3 광양시 관광안내소 광양 문화관광 1경 백운산 4대...,18988,"['광양시', '문화', '관광', '061', '797', '3', '광양시', ...","[('광양시', 'Noun'), ('문화', 'Noun'), ('관광', 'Noun..."
901124,자막보기 공지사항 제11회 최수종과 함께하 동북아 LNG Hub 광양시립소년소녀합창...,18988,"['자막', '보기', '공지', '사항', '제', '11회', '최수종', '과...","[('자막', 'Noun'), ('보기', 'Noun'), ('공지', 'Noun'..."


In [4]:
df['joined_morphs'] = df['morphs'].apply(lambda x: ' '.join(x.strip("]['").split("', '")))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs
0,25일급여 삼성카드 특판적금 서울행복신협 구즉신협 카카오뱅크8 적금 새빗켐 지대로사...,0,"['25일', '급여', '삼성', '카드', '특판', '적금', '서울', '행...","[('25일', 'Number'), ('급여', 'Foreign'), ('삼성', ...",25일 급여 삼성 카드 특판 적금 서울 행복 신협 구즉 신협 카카오 뱅크 8 적금 ...
1,근데 급 다음주부터 발령나서 다른 곳으로 출근한다,0,"['근데', '급', '다음주', '부터', '발령', '나서', '다른', '곳'...","[('근데', 'Adverb'), ('급', 'Noun'), ('다음주', 'Nou...",근데 급 다음주 부터 발령 나서 다른 곳 으로 출근 한다
2,2 3일 전에 알려주고 갑자기 거기로 출근이야 좌천인가 걱정됨,0,"['2', '3일', '전', '에', '알려주고', '갑자기', '거기', '로'...","[('2', 'Number'), ('3일', 'Number'), ('전', 'Nou...",2 3일 전 에 알려주고 갑자기 거기 로 출근 이야 좌천 인가 걱정 됨
3,잘 적응하라고 뇌물을 준비해야 하나 ㅋㅋ 갑자기 문자메세지로 삼성카드 5월 링크 이...,0,"['잘', '적응하라고', '뇌물', '을', '준비', '해야', '하나', 'ㅋ...","[('자다', 'Verb'), ('적응하다', 'Adjective'), ('뇌물',...",잘 적응하라고 뇌물 을 준비 해야 하나 ㅋㅋ 갑자기 문자 메세지 로 삼성 카드 5월...
4,소소하게 가계부 적립할게 생겨서 기분 좋은 날 2 화요일 오래전 엄마 집 옆에 있는...,0,"['소소하게', '가계부', '적립', '할게', '생겨서', '기분', '좋은',...","[('소소하다', 'Adjective'), ('가계부', 'Noun'), ('적립'...",소소하게 가계부 적립 할게 생겨서 기분 좋은 날 2 화요일 오래전 엄마 집 옆 에 ...
...,...,...,...,...,...
901121,현재 지하 1층에서 흙과몸 성긴 연결 촘촘한 관계 6 9 전시실 를 테마로 강요배 ...,18988,"['현재', '지하', '1', '층', '에서', '흙', '과', '몸', '성...","[('현재', 'Noun'), ('지하', 'Noun'), ('1', 'Number...",현재 지하 1 층 에서 흙 과 몸 성 긴 연결 촘촘 한 관계 6 9 전시 실 를 테...
901122,그 외에 Big Man 다시 일어서는 몸 박치호 애도 상실의 끝에서 김수자 게르하르...,18988,"['그', '외', '에', 'Big', 'Man', '다시', '일어서는', '몸...","[('그', 'Noun'), ('외', 'Noun'), ('에', 'Josa'), ...",그 외 에 Big Man 다시 일어서는 몸 박치 호 애도 상실 의 끝 에서 김수자 ...
901123,광양시 문화관광 061 797 3 광양시 관광안내소 광양 문화관광 1경 백운산 4대...,18988,"['광양시', '문화', '관광', '061', '797', '3', '광양시', ...","[('광양시', 'Noun'), ('문화', 'Noun'), ('관광', 'Noun...",광양시 문화 관광 061 797 3 광양시 관광 안내소 광양 문화 관광 1 경 백운...
901124,자막보기 공지사항 제11회 최수종과 함께하 동북아 LNG Hub 광양시립소년소녀합창...,18988,"['자막', '보기', '공지', '사항', '제', '11회', '최수종', '과...","[('자막', 'Noun'), ('보기', 'Noun'), ('공지', 'Noun'...",자막 보기 공지 사항 제 11회 최수종 과 함께하 동북아 LNG Hub 광양 시립 ...


In [5]:
df['joined_sentences'] = df.groupby('org_idx')['joined_morphs'].transform(lambda x: ' '.join(x))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs,joined_sentences
0,25일급여 삼성카드 특판적금 서울행복신협 구즉신협 카카오뱅크8 적금 새빗켐 지대로사...,0,"['25일', '급여', '삼성', '카드', '특판', '적금', '서울', '행...","[('25일', 'Number'), ('급여', 'Foreign'), ('삼성', ...",25일 급여 삼성 카드 특판 적금 서울 행복 신협 구즉 신협 카카오 뱅크 8 적금 ...,25일 급여 삼성 카드 특판 적금 서울 행복 신협 구즉 신협 카카오 뱅크 8 적금 ...
1,근데 급 다음주부터 발령나서 다른 곳으로 출근한다,0,"['근데', '급', '다음주', '부터', '발령', '나서', '다른', '곳'...","[('근데', 'Adverb'), ('급', 'Noun'), ('다음주', 'Nou...",근데 급 다음주 부터 발령 나서 다른 곳 으로 출근 한다,25일 급여 삼성 카드 특판 적금 서울 행복 신협 구즉 신협 카카오 뱅크 8 적금 ...
2,2 3일 전에 알려주고 갑자기 거기로 출근이야 좌천인가 걱정됨,0,"['2', '3일', '전', '에', '알려주고', '갑자기', '거기', '로'...","[('2', 'Number'), ('3일', 'Number'), ('전', 'Nou...",2 3일 전 에 알려주고 갑자기 거기 로 출근 이야 좌천 인가 걱정 됨,25일 급여 삼성 카드 특판 적금 서울 행복 신협 구즉 신협 카카오 뱅크 8 적금 ...
3,잘 적응하라고 뇌물을 준비해야 하나 ㅋㅋ 갑자기 문자메세지로 삼성카드 5월 링크 이...,0,"['잘', '적응하라고', '뇌물', '을', '준비', '해야', '하나', 'ㅋ...","[('자다', 'Verb'), ('적응하다', 'Adjective'), ('뇌물',...",잘 적응하라고 뇌물 을 준비 해야 하나 ㅋㅋ 갑자기 문자 메세지 로 삼성 카드 5월...,25일 급여 삼성 카드 특판 적금 서울 행복 신협 구즉 신협 카카오 뱅크 8 적금 ...
4,소소하게 가계부 적립할게 생겨서 기분 좋은 날 2 화요일 오래전 엄마 집 옆에 있는...,0,"['소소하게', '가계부', '적립', '할게', '생겨서', '기분', '좋은',...","[('소소하다', 'Adjective'), ('가계부', 'Noun'), ('적립'...",소소하게 가계부 적립 할게 생겨서 기분 좋은 날 2 화요일 오래전 엄마 집 옆 에 ...,25일 급여 삼성 카드 특판 적금 서울 행복 신협 구즉 신협 카카오 뱅크 8 적금 ...
...,...,...,...,...,...,...
901121,현재 지하 1층에서 흙과몸 성긴 연결 촘촘한 관계 6 9 전시실 를 테마로 강요배 ...,18988,"['현재', '지하', '1', '층', '에서', '흙', '과', '몸', '성...","[('현재', 'Noun'), ('지하', 'Noun'), ('1', 'Number...",현재 지하 1 층 에서 흙 과 몸 성 긴 연결 촘촘 한 관계 6 9 전시 실 를 테...,웰빙 Well being 과 행복 Happiness 의 합성어 웰니스 Wellnes...
901122,그 외에 Big Man 다시 일어서는 몸 박치호 애도 상실의 끝에서 김수자 게르하르...,18988,"['그', '외', '에', 'Big', 'Man', '다시', '일어서는', '몸...","[('그', 'Noun'), ('외', 'Noun'), ('에', 'Josa'), ...",그 외 에 Big Man 다시 일어서는 몸 박치 호 애도 상실 의 끝 에서 김수자 ...,웰빙 Well being 과 행복 Happiness 의 합성어 웰니스 Wellnes...
901123,광양시 문화관광 061 797 3 광양시 관광안내소 광양 문화관광 1경 백운산 4대...,18988,"['광양시', '문화', '관광', '061', '797', '3', '광양시', ...","[('광양시', 'Noun'), ('문화', 'Noun'), ('관광', 'Noun...",광양시 문화 관광 061 797 3 광양시 관광 안내소 광양 문화 관광 1 경 백운...,웰빙 Well being 과 행복 Happiness 의 합성어 웰니스 Wellnes...
901124,자막보기 공지사항 제11회 최수종과 함께하 동북아 LNG Hub 광양시립소년소녀합창...,18988,"['자막', '보기', '공지', '사항', '제', '11회', '최수종', '과...","[('자막', 'Noun'), ('보기', 'Noun'), ('공지', 'Noun'...",자막 보기 공지 사항 제 11회 최수종 과 함께하 동북아 LNG Hub 광양 시립 ...,웰빙 Well being 과 행복 Happiness 의 합성어 웰니스 Wellnes...


In [6]:
df_tf = df[['org_idx', 'joined_sentences']].drop_duplicates()
df_tf.reset_index(drop=True, inplace=True)
df_tf

,org_idx,joined_sentences
0,0,25일 급여 삼성 카드 특판 적금 서울 행복 신협 구즉 신협 카카오 뱅크 8 적금 ...
1,1,일요일 주말 휴가 가 생겨서 오빠 랑 급 안동 으로 여행 다녀왔어용 하루 여행 이라...
2,2,경주 켄싱턴 리조트 퇴실 후 곧장 집 으로 가자하면 어쩌나 했는데 다행 이 내 가 ...
3,3,안녕 재미없었고 별 다른 일이 없어서 조용했던 저번 주 와는 달리 시끌벅적 우당탕 ...
4,4,수영구 에서 뭐 재미있는 거 배울만 한 게 있을까 새로운 것 을 도전 해보면서 문화...
...,...,...
18981,18984,01 까마귀 The Raven 1943 앙리 조르주 클루 조 02 오르페브르 의 부...
18982,18985,안녕하세요 쪼 니 입니다 만약 안동 에 하루 의 시간 이 있다면 어디 를 가면 좋을...
18983,18986,선비 촌 과 더불어 입장권 한 장 으로 둘러볼 수 있는 소수서원 은 한국 최초 의 ...
18984,18987,기획 부터 문 을 열기 까지 햇수 로 6년 2000일 을 공 들 여 선보인 트리니티...


In [7]:
# TF-IDF 생성
""" CountVectorizer로 TF-IDF를 만들어서 변환하는 방법도 있지만 """
""" 처음부터 Tfidf를 구해주는 함수(TfidfVectorizer)가 있기 때문에 이를 사용할 것임. """

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# 데이터 프레임에서 분석할 문서를 가져와서 쓸 경우 꼭 리스트로 변환해서 담아줘야 함.
corpus = df_tf['joined_sentences'].to_list()

In [8]:
# stop_words에는 위에서 조사만 추출해서 만든 불용어 리스트 사용
tfidf = TfidfVectorizer(stop_words = josa_list, token_pattern=r'\w{1,}')
# tfidf_v = TfidfVectorizer(stop_words = hyeong_josa2).fit(corpus)
tdm = tfidf.fit_transform(corpus)

In [9]:
tdm.shape

(18986, 442675)

In [10]:
# 만든 TF-IDF를 데이터 프레임에 담기 위해 우선 numpy.array()로 변환함
tfidf_array = tdm.toarray()
tfidf_array

array([[0.03313881, 0.        , 0.00823036, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01103008, 0.03029482, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.0125013 , 0.03052055, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [11]:
# 변환한 array를 다시 pd.Dataframe으로 바꿈
tfidf_DF = pd.DataFrame(tfidf_array)
tfidf_DF

,0,1,2,3,4,5,6,7,8,9,...,442665,442666,442667,442668,442669,442670,442671,442672,442673,442674
0,0.033139,0.000000,0.00823,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.011030,0.030295,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.013263,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18981,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
18982,0.018205,0.050001,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
18983,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
18984,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [12]:
# 이 데이터프레임의 컬럼명은 숫자로 되어 있음. 이를 feature_names로 바꿔줘야 함.
featurenames = tfidf.get_feature_names_out()
tfidf_DF.columns = featurenames

In [13]:
tfidf_DF

,0,00,000,0001,00010,0001008,00011,00012,00013,00014,...,힛타이트,힛트곡,힛트쏭,힛트칠,힛포,힝,힝구,힝힝,힣,힣힣
0,0.033139,0.000000,0.00823,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.011030,0.030295,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.013263,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18981,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
18982,0.018205,0.050001,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
18983,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
18984,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [14]:
tfidf_DF.to_csv("../dataset/2022/본문_TF-IDF/7월/나무문화_7월_TF-IDF행렬(원본).csv", index=False, encoding='UTF-8')

In [15]:
import re
# 데이터프레임에서 한글로만 이루어진 column 필터링을 위한 정규표현식 패턴
pattern = r'^[ㄱ-ㅎㅏ-ㅣ가-힣\s]+$'

# 한글로만 이루어진 column으로 된 데이터프레임 생성
korean_df = tfidf_DF[[column for column in tfidf_DF.columns if re.match(pattern, column)]]

print(korean_df)

         ㄱ   ㄱㄱ  ㄱㄱㄱ  ㄱㄱㅎ   ㄱㄴ  ㄱㄴㄷ  ㄱㄴㄷㄹ  ㄱㄴㄹ  ㄱㄴㅁㅅㅇ  ㄱㄴㅇ  ...  힛타이트  힛트곡  \
0      0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   
1      0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   
2      0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   
3      0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   
4      0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   
...    ...  ...  ...  ...  ...  ...   ...  ...    ...  ...  ...   ...  ...   
18981  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   
18982  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   
18983  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   
18984  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   
18985  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   

       힛트쏭  힛트칠   힛포         힝   힝구   힝힝    힣   힣힣  
0      0.0

In [16]:
# 각 단어에 대한 TF-IDF의 합
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_count

,단어,tf-idf
0,0,97.314981
1,00,261.457320
2,000,37.121621
3,0001,0.385564
4,00010,0.048360
...,...,...
442670,힝,2.765905
442671,힝구,0.311839
442672,힝힝,0.110216
442673,힣,1.525258


In [17]:
# 단어 - TF-IDF 파일로 저장.
word_count.to_csv("../dataset/2022/본문_TF-IDF/7월/나무문화_7월_TF-IDF행렬합.txt", index=False, encoding='UTF-8')